In [1]:
import torch

In [2]:
torch.__version__

'1.13.0.dev20220612+cu113'

In [3]:
model_name = "google/t5-v1_1-xl"
tokenizer_name = "t5-large"

In [11]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, model_max_length=512)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [12]:
model.load_state_dict(torch.load("3B_demo.pt"))


<All keys matched successfully>

In [13]:
model.eval();

In [68]:
text = "I is reading about AI articles "
inputs = tokenizer("grammar:"+text, truncation=True, return_tensors='pt')

output = model.generate(inputs['input_ids'], num_beams=5, max_length=512, early_stopping=True)
correction=tokenizer.batch_decode(output, skip_special_tokens=True)
print("".join(correction))


I am reading about AI articles.


In [31]:
import colorama
from colorama import Fore, Back, Style
colorama.init(autoreset=True)

In [165]:
def check_grammar(input_text):
    """ takes an input sentence"""
    input_tokens = tokenizer("grammar:"+input_text, truncation=True, return_tensors="pt")
    input_tokens.to("cuda:0")
    output = model.generate(input_tokens['input_ids'], num_beams=5, max_length=512, early_stopping=True)
    correction=tokenizer.batch_decode(output, skip_special_tokens=True)
    res = "".join(correction)
    print(f"Input:  {input_text}")
    print(f"\nCorrected:  {res}")
    

In [136]:
test_incorrect_tense = "I is reading about AI articles"
#test_correct_tense = "I am reading about AI articles"
test_incorrect_tense2 = "The cherry blossoms was so beautiful"
test1 = "The kids love eating, coloring and to play with their toys"
test3 = "Janine packed early for the AWS reMARS conference. Because she was so excited to learn about FSDP."
test2 =  "My mothers cabin is next to his' cabin."

In [ ]:
model.to("cuda:0")

In [125]:
t1 = "My mothers cabin is next to his' cabin."
t2 = "Its a cold day for October."

In [ ]:
samples = [test1, test3, test2, test]

In [158]:
def model_info(in_model):
    print(f"--> Model T5 1.1")
    total_params = sum(p.numel() for p in in_model.parameters() if p.requires_grad)
    print(f"\n--> {model_name[7:]} has {round(total_params/1e9,5)} Billion params\n")

In [159]:
model_info(model)

--> Model T5 1.1

--> t5-v1_1-xl has 2.84976 Billion params



T5 Grammar Checker Demo

In [170]:
model_info(model)

--> Model T5 1.1

--> t5-v1_1-xl has 2.84976 Billion params



In [171]:
check_grammar(test1)

Input:  The kids love eating, coloring and to play with their toys

Corrected:  The kids love eating, coloring and playing with their toys.


In [172]:
check_grammar(test2)

Input:  My mothers cabin is next to his' cabin.

Corrected:  My mother's cabin is next to his cabin.


In [173]:
check_grammar(test3)

Input:  Janine packed early for the AWS reMARS conference. Because she was so excited to learn about FSDP.

Corrected:  Janine packed early for the AWS reMARS conference because she was so excited to learn about FSDP.
